# Part 1 - A description of the problem and a discussion of the background.

In stuttgart a city of the Southern Part of Germany, if someone is looking to open a restaurant near the city center, what are the best spot to open it?


# Part 2 - A description of the data and how it will be used to solve the problem

# Description of the Data

This project will rely on public data from Foursquare located in [restaurant near Stuttgart city center](https://foursquare.com/explore?mode=url&near=Stuttgart%2C%20Germany&nearGeoId=72057594040753233&q=Restaurant)

To explore and target recommended locations across different venues according to the presence of similar facilities, we will access data through FourSquare API 
interface and arrange them as a dataframe for visualization.

# Methodology section

The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

### 1. Collect Inspection Data

In this stage I'm installing and importing the libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes (uncomment this section if you did not install geopy)
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes(uncomment this section if you did not install folium)
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.



Setting my account details to develop a valid code

In [2]:
CLIENT_ID = 'Q1SCOPV1ZNAXEN2PCO4XJ0UNQIGR5N5OLLUHJZSVRQ0P1RPZ' # your Foursquare ID
CLIENT_SECRET = 'YE4AOGJXGDYQVLUEBDL2MHICNDPDDPN43XVNIS1EIPNAF21U' # your Foursquare Secret
VERSION = '20200429'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Q1SCOPV1ZNAXEN2PCO4XJ0UNQIGR5N5OLLUHJZSVRQ0P1RPZ
CLIENT_SECRET:YE4AOGJXGDYQVLUEBDL2MHICNDPDDPN43XVNIS1EIPNAF21U


#### Let's assume we are in Porto Alegre, let's get the latitude and longitude coordinates.

In [3]:
latitude = 48.7758
longitude = 9.1829


print(latitude, longitude)

48.7758 9.1829


##### Now, let's assume that what we want is a coffee shop.
So, let's define a query to search for coffee shop that is within 1000 meters near from Porto Alegre.

In [4]:
search_query = 'Restaurant'
radius = 10000
print(search_query + ' .... OK!')

Restaurant .... OK!


In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=Q1SCOPV1ZNAXEN2PCO4XJ0UNQIGR5N5OLLUHJZSVRQ0P1RPZ&client_secret=YE4AOGJXGDYQVLUEBDL2MHICNDPDDPN43XVNIS1EIPNAF21U&ll=48.7758,9.1829&v=20200429&query=Restaurant&radius=10000&limit=30'

In [6]:
results = requests.get(url).json()
results

ant',
      'pluralName': 'Restaurants',
      'shortName': 'Restaurant',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1588169810',
    'hasPerk': False},
   {'id': '4e833f550aaf371d66ab01f1',
    'name': 'Restaurant Häberle & Pfleiderer',
    'location': {'address': 'Eberhardstraße 31-33',
     'lat': 48.77373,
     'lng': 9.178524,
     'labeledLatLngs': [{'label': 'display',
       'lat': 48.77373,
       'lng': 9.178524}],
     'distance': 395,
     'postalCode': '70174',
     'cc': 'DE',
     'city': 'Stuttgart',
     'state': 'Baden-Württemberg',
     'country': 'Deutschland',
     'formattedAddress': ['Eberhardstraße 31-33',
      '70174 Stuttgart',
      'Deutschland']},
    'categories': [{'id': '4bf58dd8d48988d10d941735',
      'name': 'German Restaurant',
      'pluralName': 'German Restaurants',
      'shortName': 'German',
      'icon': {'prefix': 'https://ss3.4sqi.n

##### Let's create a dataframe from the data we collect

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,venuePage.id,location.crossStreet,location.neighborhood
0,4d1636286d1037041cfe2ebd,China-Restaurant Shanghai,"[{'id': '4bf58dd8d48988d145941735', 'name': 'C...",v-1588169810,False,Charlottenstr. 8,48.774828,9.184219,"[{'label': 'display', 'lat': 48.774828, 'lng':...",145,70182,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Charlottenstr. 8, 70182 Stuttgart, Deutschland]",NaN,NaN,NaN
1,502a3267e4b06908c87962f3,DINEA Café & Restaurant,"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",v-1588169810,False,Eberhardstr. 28,48.773449,9.176041,"[{'label': 'display', 'lat': 48.77344919999999...",567,70173,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Eberhardstr. 28, 70173 Stuttgart, Deutschland]",552069913,NaN,NaN
2,4fe9a4f1e4b0a53f6dab94f3,DINEA Café & Restaurant,"[{'id': '52e81612bcbc57f1066b79f4', 'name': 'B...",v-1588169810,False,Königstr. 6,48.781775,9.180324,"[{'label': 'display', 'lat': 48.78177547733466...",691,70173,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Königstr. 6, 70173 Stuttgart, Deutschland]",NaN,NaN,NaN
3,5730d048498e47f585347a7b,Shima Restaurant,"[{'id': '4bf58dd8d48988d1d2941735', 'name': 'S...",v-1588169810,False,Olgastr. 86,48.769887,9.180345,"[{'label': 'display', 'lat': 48.76988710000001...",684,70180,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Olgastr. 86, 70180 Stuttgart-Mitte, Deutschland]",187189891,NaN,NaN
4,4f50ee14e4b0daf96d18e98f,Restaurant WEISSENHOF,"[{'id': '4bf58dd8d48988d10d941735', 'name': 'G...",v-1588169810,False,Heilbronner Str. 21,48.788238,9.180666,"[{'label': 'display', 'lat': 48.78823806918132...",1394,70191,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Heilbronner Str. 21, 70191 Stuttgart, Deutsch...",NaN,NaN,NaN


In [8]:
dataframe.shape

(30, 19)

In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,China-Restaurant Shanghai,Chinese Restaurant,Charlottenstr. 8,48.774828,9.184219,"[{'label': 'display', 'lat': 48.774828, 'lng':...",145,70182,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Charlottenstr. 8, 70182 Stuttgart, Deutschland]",NaN,NaN,4d1636286d1037041cfe2ebd
1,DINEA Café & Restaurant,Buffet,Eberhardstr. 28,48.773449,9.176041,"[{'label': 'display', 'lat': 48.77344919999999...",567,70173,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Eberhardstr. 28, 70173 Stuttgart, Deutschland]",NaN,NaN,502a3267e4b06908c87962f3
2,DINEA Café & Restaurant,Buffet,Königstr. 6,48.781775,9.180324,"[{'label': 'display', 'lat': 48.78177547733466...",691,70173,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Königstr. 6, 70173 Stuttgart, Deutschland]",NaN,NaN,4fe9a4f1e4b0a53f6dab94f3
3,Shima Restaurant,Sushi Restaurant,Olgastr. 86,48.769887,9.180345,"[{'label': 'display', 'lat': 48.76988710000001...",684,70180,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Olgastr. 86, 70180 Stuttgart-Mitte, Deutschland]",NaN,NaN,5730d048498e47f585347a7b
4,Restaurant WEISSENHOF,German Restaurant,Heilbronner Str. 21,48.788238,9.180666,"[{'label': 'display', 'lat': 48.78823806918132...",1394,70191,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Heilbronner Str. 21, 70191 Stuttgart, Deutsch...",NaN,NaN,4f50ee14e4b0daf96d18e98f
5,Persian Restaurant,Restaurant,Kanalstr. 10,48.775200,9.183307,"[{'label': 'display', 'lat': 48.77520008324453...",73,70182,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Kanalstr. 10, 70182 Stuttgart, Deutschland]",NaN,NaN,4d5964173358a090b4058844
6,Restaurant Häberle & Pfleiderer,German Restaurant,Eberhardstraße 31-33,48.773730,9.178524,"[{'label': 'display', 'lat': 48.77373, 'lng': ...",395,70174,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Eberhardstraße 31-33, 70174 Stuttgart, Deutsc...",NaN,NaN,4e833f550aaf371d66ab01f1
7,Restaurant Adulis II,German Restaurant,NaN,48.774264,9.178367,"[{'label': 'display', 'lat': 48.77426375431874...",373,NaN,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Stuttgart, Deutschland]",NaN,NaN,4c4afea09e6dbe9a202e610b
8,Restaurant Saigon,None,NaN,48.773671,9.178558,"[{'label': 'display', 'lat': 48.773671, 'lng':...",397,NaN,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Stuttgart, Deutschland]",NaN,NaN,4b79362ff964a52035ef2ee3
9,Viertel Restaurant & Weinstube,Mediterranean Restaurant,Schubartstr. 29,48.786890,9.194910,"[{'label': 'display', 'lat': 48.78689, 'lng': ...",1516,70190,DE,Stuttgart,Baden-Württemberg,Deutschland,"[Schubartstr. 29 (Werastr), 70190 Stuttgart, D...",Werastr,Ostheim,5abfd183be70786dd63ed1bc


In [39]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) #map centered in karlsruhe

folium.CircleMarker(
    [latitude, longitude],
    radius=10000,
    color='red',
    fill = False,
).add_to(venues_map)

for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map

### These points are the restaurant nearby stuttgar city center in Germany.

##### We are going to use the above data in the following way:
Knowing the locations of existing restaurants, we can apply Kernel Density Estimation (KDE) to determine the area of influence of the existing cafeterias and discovery where someone who wants can install its own new restaurants.